<a href="https://colab.research.google.com/github/YujiSue/IJPlugIns/blob/master/ImageJ_Plugin_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ImageJ plugin compiler


## Single source file => JAVA class format plugin

In [ ]:
#@title 1. Download IJ source and install OpenJDK (1.8)
!git clone https://github.com/imagej/ImageJ.git
!sudo apt-get install -y openjdk-8-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
!javac -version

In [ ]:
#@title 2. Upload source
#@markdown Upload your plugin source file "xxx_yyyy.java" into the directory [/content/ImageJ/plugins].
import os
from google.colab import files
res = files.upload()
os.system('mv -f /content/' + list(res)[0] + ' /content/ImageJ/plugins/')

In [ ]:
#@title 3. Compile java source to make a class file
#@markdown Copy and paste your plugin's source file name.
import os
plugin_file = 'My_Plugin.java' #@param {"type":"string"}
os.system('javac -sourcepath /content/ImageJ /content/ImageJ/plugins/' + plugin_file)

In [ ]:
#@title 4. Download the compiled class file
#@markdown Download and install the compiled file (.class) as a plugin on your ImageJ app.
from google.colab import files
res = files.download('/content/ImageJ/plugins/'+plugin_file.replace('.java','.class'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Multiple source files => JAR format plugin

In [ ]:
#@title Step 1. Install Maven
!sudo apt-get install maven

### Step 2. Make a new project
If you are building a new plugin, perform the following steps.  
Otherwise, if you have a repository for building the plugin, skip to step 3.

In [ ]:
#@title 2-1. Make plugin project (Maven)
import os
from google.colab import userdata
%cd /content

# Init.
#imagej_version = 1 #@param [1,2]

#@markdown ### Plugin information
name = 'My Plugin' #@param {'type': 'string'}
version = '0.1.0' #@param {'type': 'string'}
os.environ['plugin_version'] = version
description = 'Sample plugin.' #@param {'type': 'string'}
develop_year = '2023' #@param {'type': 'string'}
repository = 'https://github.com/YujiSue/IJPlugIns' #@param {'type': 'string'}
os.system('git clone ' + repository)
repo_name = os.path.split(repository)[1]
license = 'CC0' #@param {'type': 'string'}
plugin_path = name.lower().replace(" ", "_")
plugin_class = name.replace(" ", "_")
os.environ['plugin_class'] = plugin_class
#plugin_subdir = '' #@param {'type': 'string'}
plugin_dir = os.path.join('/content', repo_name, plugin_path)
os.environ['plugin_dir'] = plugin_dir

#@markdown ### Author information
author = 'Yuji Suehiro' #@param {'type': 'string'}
github_id = 'YujiSue' #@param {'type': 'string'}
github_url = 'https://github.com/YujiSue' #@param {'type': 'string'}
organization = 'ys-ebase' #@#param {'type': 'string'}
org_url = 'https://ys-ebase.net/' #@#param {'type': 'string'}

#@markdown ### About source codes
group_name1 = 'net' #@param {'type': 'string'}
group_name2 = 'ysij' #@param {'type': 'string'}
package_name = '.'.join([group_name1, group_name2, 'imagej'])
full_class = '.'.join([package_name, plugin_class])

# Make dirs
source_dir = os.path.join(plugin_dir, 'src', 'main', 'java', group_name1, group_name2, 'imagej')
os.makedirs(source_dir, exist_ok=True)
os.makedirs(os.path.join(plugin_dir, 'src', 'main', 'resources'), exist_ok=True)

# Make config.
with open(os.path.join(plugin_dir, 'src', 'main', 'resources', 'plugins.config'), 'w') as f:
  f.write('Plugins, "' + name + '", ' + '.'.join([group_name1, group_name2, 'imagej', plugin_class]))

# Make pom for Maven
# Download template
!wget https://raw.githubusercontent.com/YujiSue/IJPlugIns/master/template/pom_template.xml
pom = ''
with open('/content/pom_template.xml') as f:
  pom = f.read()
pom = pom.replace('%_GROUP_%', '.'.join([group_name1, group_name2]))
pom = pom.replace('%_CLASS_%', plugin_class)
pom = pom.replace('%_VERSION_%', version)
pom = pom.replace('%_NAME_%', name)
pom = pom.replace('%_REPOSITORY_%', repository)
pom = pom.replace('%_DESCRIPTION_%', description)
pom = pom.replace('%_YEAR_%', develop_year)
pom = pom.replace('%_LICENSE_%', license)
pom = pom.replace('%_ORG_%', organization)
pom = pom.replace('%_ORG_URL_%', org_url)
pom = pom.replace('%_AUTHOR_ID_%', github_id)
pom = pom.replace('%_AUTHOR_%', author)
pom = pom.replace('%_AUTHOR_URL_%', github_url)
pom = pom.replace('%_PACKAGE_%', package_name)
pom = pom.replace('%_FULL_CLASS_%', full_class)

with open(os.path.join(plugin_dir, 'pom.xml'), 'w') as f:
  f.write(pom)

In [2]:
#@title 2-2. Generate main class source file
import os
plugin_type = 'plugin' #@param['plugin', 'filter', 'frame', 'tool']

template_file = ''
if plugin_type == 'plugin':
  os.system('wget https://raw.githubusercontent.com/YujiSue/IJPlugIns/master/template/My_Plugin.java')
  template_file = 'My_Plugin.java'
elif plugin_type == 'filter':
  os.system('wget https://raw.githubusercontent.com/YujiSue/IJPlugIns/master/template/Filter_Plugin.java')
  template_file = 'Filter_Plugin.java'
elif plugin_type == 'frame':
  os.system('wget https://raw.githubusercontent.com/YujiSue/IJPlugIns/master/template/Plugin_Frame.java')
  template_file = 'Plugin_Frame.java'
else:
  os.system('wget https://raw.githubusercontent.com/YujiSue/IJPlugIns/master/template/Prototype_Tool.java')
  template_file = 'Prototype_Tool.java'

main_src = ''
with open(template_file) as f:
  main_src = f.read()
main_src = main_src.replace('%_PACKAGE_%', package_name)
main_src = main_src.replace('%_CLASS_%', plugin_class)
main_src_path = os.path.join(source_dir, plugin_class+'.java')
with open(main_src_path, 'w') as fw:
  fw.write(main_src)

In [ ]:
#@title 2-3. Push to Github
#@markdown If need, edit your main class and append additional source file(s).<br>
#@markdown Then, push the source file(s) to Github repository.
import os
from google.colab import userdata
auth_repos = repository.replace('github.com', github_id + ":" + userdata.get('git_auth') + '@github.com')
repos = os.path.split(repository)[1]
os.system('cp -r ' + plugin_dir + ' ' + repos)
os.chdir(repo_name)
os.system('git config user.email "' +  userdata.get('git_email') + '"')
os.system('git config user.name "' +  userdata.get('git_user') + '"')
os.system('git add ' + plugin_path)
os.system('git commit -m "IJ plugin source(s) created by Google Colab."')
os.system('git remote set-url origin "' + auth_repos + '"')
os.system('git push origin master')

### Step 3. Copy project from Github
If you created the project in step 2, skip this step.

In [2]:
#@title Clone git repository
import os
import xml.etree.ElementTree as ET
repository = 'https://github.com/YujiSue/IJPlugIns' #@param {'type': 'string' }
plugin_path = 'my_plugin' #@param {'type': 'string' }
os.system('git clone ' + repository)
repo_name = os.path.splitext(os.path.split(repository)[1])[0]
plugin_dir = os.path.join('/content', repo_name, plugin_path)
os.environ['plugin_dir'] = plugin_dir
tree = ET.parse(os.path.join(plugin_dir, 'pom.xml'))
root = tree.getroot()
plugin_class = root.find('{*}artifactId').text
os.environ['plugin_class'] = plugin_class
version = root.find('{*}version').text
os.environ['plugin_version'] = version


### Build plugin

In [ ]:
#@title Step 4. Build JAR format plugin
%cd $plugin_dir
!mvn package
!cp -f $plugin_dir/target/$plugin_class-$plugin_version.jar /content/$plugin_class.jar

In [4]:
#@title Step 5. Download JAR format plugin
import os
from google.colab import files
os.chdir('/content')
res = files.download(plugin_class+'.jar')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>